In [13]:
from metafid.deriv import Pricing, OptionStrategy
from metafid.data import TSETMC
import pandas as pd
import numpy as np
import jdatetime as jdt
import requests
from collections import namedtuple
from itertools import combinations
from operator import add
import finpy_tse as fpy
import psycopg
import re
from CONFIG import DB_PASS as PASS
import inspect
from metafid.mfw import DB
import pickle
from metafid.mfw.deriv import OptionStrategyMFW

In [14]:
DBNAME = "metafid"
USER = "postgres"

ostg_mfw = OptionStrategyMFW(dbname=DBNAME, user=USER, pass_=PASS, ua_table="sigma", ostg_table="derivs_optionstrategy", pct_daily_cp=0.3, interval=10)

In [3]:
db = DB(dbname="metafid", user="postgres", pass_=PASS)

In [13]:
"https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/"

'https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/'

In [4]:
def same_strike_and_ex_date_on_call_put(call, put):
    cols = [i for i in call.columns if not i.startswith("ua_") and i not in ["t", "sigma", "dt", "type"]]
    def cols_(x):
        if (x.endswith("_x")) and (x.startswith("ua_")):
            return x.replace("_x","")
        elif (x.endswith("_x")):
            return "call_" + x.replace("_x","")
        elif (x.endswith("_y")):
            return "put_" + x.replace("_y","")
        else:
            return x
    df = call.merge(put[cols], on=["ua", "strike_price", "ex_date"], how="inner")
    df.columns = list(map(cols_, df.columns))
    return df


In [35]:
class OptionStrategy:
    def __init__(self, call:pd.DataFrame, put:pd.DataFrame, pct_daily_cp:float=None) -> None:
            self.call = call
            self.put = put
            self.pct_daily_cp = pct_daily_cp
            self.rep_df = pd.DataFrame(columns=["strategy","position","ua", "ua_final", "t", "bs", "max_pot_profit", "max_pot_loss", "break_even", "current_profit", "pct_cp", "pct_daily_cp", "evaluation"]) 
    def same_strike_and_ex_date_on_call_put(self):
        cols = [i for i in self.call.columns if not i.startswith("ua_") and i not in ["t", "sigma", "dt", "type"]]
        def cols_(x):
            if (x.endswith("_x")) and (x.startswith("ua_")):
                return x.replace("_x","")
            elif (x.endswith("_x")):
                return "call_" + x.replace("_x","")
            elif (x.endswith("_y")):
                return "put_" + x.replace("_y","")
            else:
                return x
        df = self.call.merge(self.put[cols], on=["ua", "strike_price", "ex_date"], how="inner")
        df.columns = list(map(cols_, df.columns))
        return df
    
    def rep_columns(self, cols):
         return [i for i in cols if i in self.rep_df.columns]
    
    def position(self, df):
        def replace_(t:str, char):
            for i in char:
                t = t.replace(i, "")
            return t
        df[["strategy","position"]] = df.stg.map(str).str.split("(", expand=True)
        df["position"] = df.position.apply(lambda x: replace_(x, ["'", ")"]))
        return df


    def long_straddle(self, df):
        """
        A long straddle strategy is an options strategy that involves buying a call and a put on the same
        underlying asset with the same strike price and expiration date. The strike price is usually at-the-money
        or close to it. The goal of this strategy is to profit from a very strong move in either direction by the
        underlying asset, often triggered by a newsworthy event.

        :param df: with (strike_price, ua_final, call_buy_price, put_buy_price) columns
        :return:maximum loss: net premium received
                maximum profit: unlimited
                lower break-even: strike price – net premium
                upper break-even: strike price  + net premium
        """
        stg = namedtuple("LongStraddle", "buy buy_at buy_ buy_at_")
        df["stg"] = df.apply(lambda x: stg(buy=x["call_option"], buy_at=x["call_sell_price"], buy_=x["put_option"], buy_at_=x["put_sell_price"]),
                             axis=1)
        df = self.position(df)
        
        df = df.assign(max_pot_loss=-df.call_sell_price - df.put_sell_price,
                       max_pot_profit = None)
        df = df.assign(
            lower_break_even=df.strike_price + df.max_pot_loss,
            upper_break_even=df.strike_price - df.max_pot_loss,
        )
        df["break_even"] = df.apply(lambda x: f"l: {x['lower_break_even']}, u: {x['upper_break_even']}", axis=1)
        df["bs"] = df.apply(lambda x: f"c: {x['call_bs']}, p: {x['put_bs']}", axis=1)
        df["current_profit"] = df.apply(
            lambda x: max(
                x["max_pot_loss"],
                abs(x["ua_final"] - x["strike_price"]) + x["max_pot_loss"],
            ),
            axis=1,
        )
        df = df.assign(pct_cp=df.current_profit / (df.call_sell_price + df.put_sell_price) * 100).round(2)
        df = df.assign(pct_daily_cp=df.pct_cp / df.t).round(2)
        df = df.assign(evaluation = None)
        df = df[self.rep_columns(df.columns)]

        if self.pct_daily_cp:
            return df[df.pct_daily_cp > self.pct_daily_cp]
        else:
            return df

    

In [15]:
mw = TSETMC().mw()

In [16]:

call = ostg_mfw.data().call
put = ostg_mfw.data().put

In [36]:
test = OptionStrategy(call=call, put=put, pct_daily_cp=-1)
mixed_df = test.same_strike_and_ex_date_on_call_put()
mixed_df = mixed_df[(mixed_df.call_sell_price >0) & (mixed_df.put_sell_price >0)]
mixed_df

,call_option,call_name,call_final,call_value,call_ob_depth,call_sell_vol,call_sell_price,call_buy_price,call_buy_vol,dt,ua,strike_price,ex_date,ua_name,ua_final,ua_value,ua_ob_depth,ua_sell_vol,ua_sell_price,ua_buy_price,ua_buy_vol,ua_dt,t,type,sigma,call_bs,put_option,put_name,put_final,put_value,put_ob_depth,put_sell_vol,put_sell_price,put_buy_price,put_buy_vol,put_bs
22,ضملی3023,اختیارخ فملی-5333-1402/03/07,2700,222750000,1,1,3399,2400,2,1402-01-22 01:02:50,فملی,5333,1402-03-07,ملی‌ صنایع‌ مس‌ ایران‌,8050,1676502825470,1,28560,7940,7930,39773,1402-01-22 01:02:50,47,call,0.025490,2885,طملی3023,اختیارف فملی-5333-1402/03/07,50,750000,1,90,50,1,1000,0
23,ضملی3023,اختیارخ فملی-5333-1402/03/07,2700,222750000,2,50,3400,2101,14,1402-01-22 01:02:50,فملی,5333,1402-03-07,ملی‌ صنایع‌ مس‌ ایران‌,8050,1676502825470,1,28560,7940,7930,39773,1402-01-22 01:02:50,47,call,0.025490,2885,طملی3023,اختیارف فملی-5333-1402/03/07,50,750000,1,90,50,1,1000,0
24,ضملی3023,اختیارخ فملی-5333-1402/03/07,2700,222750000,3,15,3500,1800,1,1402-01-22 01:02:50,فملی,5333,1402-03-07,ملی‌ صنایع‌ مس‌ ایران‌,8050,1676502825470,1,28560,7940,7930,39773,1402-01-22 01:02:50,47,call,0.025490,2885,طملی3023,اختیارف فملی-5333-1402/03/07,50,750000,1,90,50,1,1000,0
27,ضستا2014,اختیارخ شستا-1100-1402/02/13,329,17582199000,2,1,317,287,156,1402-01-22 01:02:50,شستا,1100,1402-02-13,سرمایه گذاری تامین اجتماعی,1382,2900261517422,1,156864,1357,1357,100000,1402-01-22 01:02:50,22,call,0.023057,298,طستا2014,اختیارف شستا-1100-1402/02/13,1,0,1,100,30,0,0,0
29,ضستا2014,اختیارخ شستا-1100-1402/02/13,329,17582199000,4,1000,349,244,1,1402-01-22 01:02:50,شستا,1100,1402-02-13,سرمایه گذاری تامین اجتماعی,1382,2900261517422,1,156864,1357,1357,100000,1402-01-22 01:02:50,22,call,0.023057,298,طستا2014,اختیارف شستا-1100-1402/02/13,1,0,1,100,30,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,ضترا3024,اختیارخ شتران-4000-1402/03/28,1673,2910510000,2,32,1680,1250,25,1402-01-22 01:02:50,شتران,4000,1402-03-28,پالایش نفت تهران,5341,2816355456770,1,350715,5240,5238,43652,1402-01-22 01:02:50,68,call,0.027585,1525,طترا3024,اختیارف شتران-4000-1402/03/28,310,0,1,9,300,0,0,2
923,ضترا3024,اختیارخ شتران-4000-1402/03/28,1673,2910510000,2,32,1680,1250,25,1402-01-22 01:02:50,شتران,4000,1402-03-28,پالایش نفت تهران,5341,2816355456770,1,350715,5240,5238,43652,1402-01-22 01:02:50,68,call,0.027585,1525,طترا3024,اختیارف شتران-4000-1402/03/28,310,0,3,12,350,0,0,2
927,ضترا3028,اختیارخ شتران-6000-1402/03/28,500,113000000,3,140,1000,111,1,1402-01-22 01:02:50,شتران,6000,1402-03-28,پالایش نفت تهران,5341,2816355456770,1,350715,5240,5238,43652,1402-01-22 01:02:50,68,call,0.027585,31,طترا3028,اختیارف شتران-6000-1402/03/28,942,5650000,1,6,1600,902,15,417
937,ضترا3028,اختیارخ شتران-6000-1402/03/28,500,113000000,1,24,500,450,20,1402-01-22 01:02:50,شتران,6000,1402-03-28,پالایش نفت تهران,5341,2816355456770,1,350715,5240,5238,43652,1402-01-22 01:02:50,68,call,0.027585,31,طترا3028,اختیارف شتران-6000-1402/03/28,942,5650000,1,6,1600,902,15,417


In [38]:
pd.set_option('display.max_colwidth', None)
df = test.long_straddle(mixed_df).sort_values("pct_daily_cp")

In [40]:
db.insert_data(table="derivs_optionstrategy", df=df)